# pySAMBUCA Main Call



- Utility functions used to populate dictionaries and arrays required to be passed to core SAMBUCA inversion algorithm
- Combination of hardcoded and local path calls are currently required, with the 'end goal' of defining XMLs to pass straight to these dictoniaries, dervied by any API/method (manual, SNAP GUI, AGDC API etc)
- Processing and Output options are hardcoded in corresponding modules as noted.

# Set up the environment



In [ ]:
import numpy as np
import sys

#import Marcos script modules and set up path to these

sys.path.append('C:\\Users\\PCUSER\\sambuca_project\\sen2coral')

import sambuca_input_rrs
import sambuca_input_parameters
import sambuca_preparation
import sambuca_calculations
import sambuca_outputs
import define_outputs


# set some controls on numpy formatting
# 5 decimal places, suppress scientific notation
np.set_printoptions(precision=5, suppress=True)



Now import the sambuca and sambuca-core packages:

In [ ]:
import sambuca as sb
import sambuca_core as sbc

In [ ]:
# Main statement needed if exporting to .py script. 
#if __name__=='__main__':

# Define a base project input path

In this path the code expects the following folders
- image
- siop
- nedr
- substrates
- sensor_filters

This path can used to point to XML files to ingest when ready for that. At the moment, files specified in the input folders are hardcoded in sambuca_input_rrs and sambuca_input_parameters modules.

In [ ]:
base_path = 'C:\\Users\\PCUSER\\sambuca_project\\sambuca\\input_data\\'

# Load the Input Image Data

- The input data is loaded into an array using rasterio. 
- Other image metadata is loaded into a tagged image_info dictionary based on rasterio src class. (e.g crs, affine, width, height)
- Full sensor filter dictonary is loaded, and specific sensor filter extracted based on sensor id key
- If Rrs is set to True, the user is supplying above surface remote sensing reflectance. This is coverted to below surface rrs if True is set. Default value if not supplied is False

File names and sensor id key are currently hardcoded.

In [ ]:
[observed_rrs, image_info]=sambuca_input_rrs.sam_obs(base_path, Rrs = False)

# Load the Parameters

Builds Dictionaries to pass to the data prep and inversion 
- SIOPs: spectral, values, substrates, free parameter bounds
- Enviromental data: sun angle. off nadir, q factor

Currently hardcoded and read in from input data folder, aim to build from XML


In [ ]:
[siop, envmeta]=sambuca_input_parameters.sam_par(base_path)

# Prepare the Data

- Prepares the spectral inputs (currently truncates to a common wavelength range).
- Builds a fixed parameter set out of truncated inputs, siop and envdata dictionaries.
- Prepares a result recorder for storing array based results.
- Defines the objective function (Hardcoded call currently made to specific error function here)

In [ ]:
[wavelengths, siop, image_info, fixed_parameters, result_recorder, objective]=sambuca_preparation.sam_prep(siop, envmeta, 
                                                                                                           image_info)

# Run the Inversion

Defined in sambuca_calculations:
- Type of SciPy optimisation used
- Definition of an image subset to be processed if required (primarily for testing; as full result recorder array extent of original image is still used, and crs/affine may not translate when written to geotiff in outputs)
- Parameter starting points (currently mid point of range)
- Parallel processing option for substrate pairs
- Option for SAMBUCA option of 'go shallow', when set to true, retrieves as shallow depth as possible for optically deep waters whilst maintaining an SDI value of < 1.

Returns the results, spatial subset range processed, and number of pixels processed

In [ ]:
[result_recorder, coordinates, num_pixels]=sambuca_calculations.sam_com(observed_rrs, objective, siop,
                                                                        result_recorder, image_info, shallow = True)

# Define and Write some Outputs

- Outputs types wanted are defined in define_outputs.py and call to a writeout function within sambuca_outputs.py
- Written out as geotiffs with crs info inherited from the input rrs data
- If a subsection of image to be processed was selected in sambuca_calculations, coordinates ensure only this section is written (validation needed to see if crs and affine info translates correctly for a subset write)

In [ ]:
define_outputs.output_suite(result_recorder, image_info, coordinates)